In [1]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [2]:
import numpy as np
import tensorflow as tf
import os
import cv2
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image


train_dir = r'E:\AI\dataset_skeleton_for_generator\training'
val_dir = r'E:\AI\dataset_skeleton_for_generator\validation'
test_dir = r'E:\AI\dataset_skeleton_for_generator\test'

# ImageDataGenerator 초기화
datagen = ImageDataGenerator(rescale=1./255)  # 이미지를 0과 1 사이의 값으로 정규화

# 훈련 데이터셋을 위한 제너레이터 생성
train_generator = datagen.flow_from_directory(
        train_dir,  # 훈련 데이터셋 디렉토리
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical')

validation_generator = datagen.flow_from_directory(
        val_dir,  # 훈련 데이터셋 디렉토리
        target_size=(128, 128), 
        batch_size=32,
        class_mode='categorical') 

print(train_generator.samples)
print(train_generator.num_classes)


Found 709960 images belonging to 269 classes.
Found 130596 images belonging to 269 classes.
709960
269


In [ ]:

# 모델 정의
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,  # 검증 데이터셋 설정
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


Epoch 1/10
22186/22186 [==============================] - 2748s 124ms/step - loss: 2.9949 - accuracy: 0.1456 - val_loss: 5.4176 - val_accuracy: 0.1416
Epoch 2/10
14357/22186 [==================>...........] - ETA: 9:22 - loss: 2.5690 - accuracy: 0.1970

In [ ]:
model.summary()

In [ ]:
# 테스트 데이터셋 디렉토리 경로 설정


# ImageDataGenerator 초기화 (테스트 데이터셋용)
test_datagen = ImageDataGenerator(rescale=1./255)  # 이미지를 0과 1 사이의 값으로 정규화

# 테스트 데이터셋을 위한 제너레이터 생성
test_generator = test_datagen.flow_from_directory(
        test_dir,  # 테스트 데이터셋 디렉토리
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical',  # 다중 분류 문제인 경우 'categorical'
        shuffle=False)  # 평가 시 데이터 순서를 유지

# 모델 성능 평가
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)

print('테스트 정확도:', test_acc)
